In [0]:
!curl https://colab.chainer.org/install | sh -
!apt-get install graphviz
!pip install 'chaineripy'
!pip install 'chainercv'

import sys
import os
from functools import partial
import json


from chainer.links.caffe.caffe_function import CaffeFunction
from chainer.serializers import npz
from chainer.datasets import cifar, split_dataset_random, TransformDataset
from chainer import links as L
from chainer import initializers, iterators
from chainer.training import extensions
from google.colab import drive
import numpy as np


base_dir = './drive/My Drive/study'
sys.path.append(base_dir)
drive.mount('/content/drive')

caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezenet_v1.1.caffemodel')
npz.save_npz('squeezenet_v1_1.npz', caffemodel, compression=False)
caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezeNet_residual.caffemodel')
npz.save_npz('squeeze_resnet.npz', caffemodel, compression=False)

from squeezenet.networks.squeezenet import SqueezeNet
from squeezenet.networks.squeezenet_bn import SqueezeNetBN
from squeezenet.networks.squeeze_resnet import SqueezeResNet
from squeezenet.networks.squeeze_pre_resnet import SqueezePreResNet
from execute import run_train
from transform import transform

# Dataset setup
train_val, test = cifar.get_cifar10(scale=255.)
train_size      = int(len(train_val) * 0.9)
train, valid    = split_dataset_random(train_val, train_size, seed=0)

mean = np.mean([x for x, _ in train], axis=(0, 2, 3))
std  = np.std([x for x, _ in train], axis=(0, 2, 3))

params = json.load(open(base_dir + '/params.json', "r"))
params['output_size'] = (227, 227)
params['batchsize']     = 128

In [0]:
# Data Augmentation
transform_train = partial(transform, mean=mean, std=std, train=True, **params)
transform_valid = partial(transform, mean=mean, std=std, **params)

processed_train = TransformDataset(train, transform_train)
processed_valid = TransformDataset(valid, transform_valid)
train_iter = iterators.SerialIterator(processed_train, params['batchsize'])
valid_iter = iterators.SerialIterator(processed_valid, params['batchsize'], repeat=False, shuffle=False)

# Model
model = SqueezeNet(10, pretrained_model='squeezenet_v1_1.npz', init_param=initializers.HeNormal())
net = L.Classifier(model)

# Evaluator
evaluator = extensions.Evaluator(valid_iter, net, device=params['gpu_id'])

# RUN!

params['changed_layer'] = [net.predictor.conv10.W, net.predictor.conv10.b]
params['freeze_layer']  = ['base/conv1', 'base/fire2', 'base/fire3', 'base/fire4', 'base/fire5']
params['result_dir']    = 'drive/My Drive/study/result'
run_train(train_iter, net, evaluator, **params)

/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:524: UserWarning: Hook timing attribute not in ('pre', 'post'), defaulting timing to 'pre'.
  warnings.warn("Hook timing attribute not in ('pre', 'post'), "


epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  lr          elapsed_time
1           1.99018     0.258856       1.33413        0.508008           0.001       165.382       
2           1.22004     0.559592       0.834563       0.705273           0.001       331.161       
3           0.960058    0.661214       0.706628       0.75957            0.001       496.66        
4           0.819807    0.713312       0.632224       0.780859           0.001       662.574       
5           0.729975    0.74448        0.558954       0.804102           0.001       828.159       
6           0.667131    0.766757       0.532131       0.819141           0.001       994.858       
7           0.615927    0.785056       0.475573       0.838477           0.001       1160.41       
